# Data Cleaning - fullstat

Description: 
- fullstat split into to fullstat_labeled and fullstat_without
- cleaned fullstat for topic modelling and engagement score modelling

## Import Libraries

### Main Libraries

In [1]:
import pandas as pd
import numpy as np
import re

### NLP Libraries

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Gensim
from gensim.parsing.porter import PorterStemmer

## Load Dataset (fullstat_labeled)

In [3]:
# Load dataset
df = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/fullstat_labeled.tsv', sep='\t')
# Inspect df
df.head()

,Unnamed: 0,by,category,comment_likes_count,comments_base,comments_count_fb,comments_replies,comments_retrieved,engagement_fb,full_picture,...,post_published_unix,rea_ANGRY,rea_HAHA,rea_LOVE,rea_SAD,rea_THANKFUL,rea_WOW,reactions_count_fb,shares_count_fb,type
0,0,post_page_155027942462,App Update,0,0,11,0,0,22,https://scontent.xx.fbcdn.net/v/t1.0-9/p720x72...,...,1561645813,0,0,0,0,0,0,11,0,photo
1,1,post_page_155027942462,Engagement,0,0,30,0,0,110,https://scontent.xx.fbcdn.net/v/t1.0-9/p720x72...,...,1561127466,1,0,4,0,0,1,76,4,photo
2,2,post_page_155027942462,Engagement,0,0,11,0,0,17,https://scontent.xx.fbcdn.net/v/t1.0-9/p720x72...,...,1561042828,1,0,0,0,0,0,6,0,photo
3,3,post_page_155027942462,Engagement,0,0,10,0,0,19,https://scontent.xx.fbcdn.net/v/t1.0-9/p720x72...,...,1560781811,1,0,0,0,0,0,9,0,photo
4,4,post_page_155027942462,Engagement,0,0,8,0,0,17,https://external.xx.fbcdn.net/safe_image.php?d...,...,1560522602,3,0,0,0,0,0,9,0,video


## Data Pre-processing

In [4]:
# Drop unnecessary columns
drop_columns = ['Unnamed: 0', 'post_link', 'picture', 'full_picture', 
                'link', 'link_domain', 'post_published_unix', 'post_published_sql']
df = df.drop(drop_columns, axis=1)

In [5]:
# Tokenize the posts
df['post_message'] = df['post_message'].apply(lambda list_words: word_tokenize(list_words))

In [6]:
def delete_url(text):
    '''
    To delete rows that looks like a URL
    (https, http, www)
    '''
    return re.sub(r'''(?i)\b((?:http[s]?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', ' ', text)

In [7]:
# Run the function to remove non-letter characters
df['post_message'] = df['post_message'].apply(lambda list_words: [delete_url(word) for word in list_words])

In [8]:
def remove_nonalpha(text):
    '''
    Removing non-alpha characters
    '''
    return re.sub('[^a-zA-Z]', '', text)

In [9]:
# Run the function to remove non-letter characters
df['post_message'] = df['post_message'].apply(lambda list_words: [remove_nonalpha(word) for word in list_words])

In [10]:
# Remove empty strings
df['post_message'] = df['post_message'].apply(lambda list_words: list(filter(None, list_words)))

In [11]:
# Convert all letters to lowercase
df['post_message'] = df['post_message'].apply(lambda list_words: [word.lower() for word in list_words])

In [12]:
# Remove stopwords
stop_words = sorted(stopwords.words('english'))
stop_words.append('http')
df['post_message'] = df['post_message'].apply(lambda list_words: [word for word in list_words if not word in stop_words])

In [13]:
# Sort words by grouping inflected forms of the same word
lem = WordNetLemmatizer()
df['post_message'] = df['post_message'].apply(lambda list_words: [lem.lemmatize(word) for word in list_words])

In [14]:
# Eliminate affixes using gensim's PorterStemmer
p = PorterStemmer()
df['post_message'] = df['post_message'].apply(lambda list_words: [p.stem(word) for word in list_words])

In [15]:
# convert list to string
df['post_message'] = df['post_message'].apply(lambda row: ' '.join(row))

## Save as fullstat_cleaned.tsv

In [16]:
df.to_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/cleaned/fullstatCleaned_withLabels.tsv', sep='\t')